In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [20]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [21]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-1 labels=frozenset({'User'}) properties={'indexes': [], 'name': 'User', 'constraints': []}>] relationships=[<Relationship id=-1 nodes=(<Node id=-1 labels=frozenset({'User'}) properties={'indexes': [], 'name': 'User', 'constraints': []}>, <Node id=-1 labels=frozenset({'User'}) properties={'indexes': [], 'name': 'User', 'constraints': []}>) type='IS_FRIENDS_WITH' properties={}>]>]

In [5]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET LIBRARY

In [12]:
import numpy as np
import pandas as pd

/Users/boO/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
## SET WORK PATH

In [13]:
import os

In [14]:
#work_path = '/home/dmig/work/this-book/neo4j-book'
work_path = '/Users/boO/data8/data.student/neo4j-book/neo4j-book'
os.chdir(work_path)

In [18]:
tbl = pd.read_csv("{}/data/train.csv".format(work_path))

In [ ]:
tbl.shape

In [ ]:
tbl.head()

In [ ]:
node_id = np.union1d(tbl.source_node.unique(), tbl.destination_node.unique())
# len(node_id) # 1,862,220
#node_id = np.random.choice(node_id, size=100000, replace=False)
node_id = pd.DataFrame(node_id, columns=['id:ID'])
node_id[':LABEL'] = 'User'
node_id.to_csv("{}/data/node.csv".format(work_path), index=False)

In [16]:
tbl.columns = [':START_ID',':END_ID']
tbl[':TYPE'] = 'IS_FRIENDS_WITH'
tbl.to_csv("{}/data/edge.csv".format(work_path), index=False)

In [ ]:
## LOAD DATA

In [ ]:
!bin/neo4j-admin import --nodes=import/node.csv --relationships=import/edge.csv

In [ ]:
## IT NEVER ENDS
gph_conn.query("""
               // USING PERIODIC COMMIT 500
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MERGE (:User {id: row.source_node})
               """)
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MERGE (:User {id: row.destination_node})
               """)
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MATCH (s:User {id: row.source_node}) 
               MATCH (d:User {id: row.destination_node}) 
               MERGE (s)-[:IS_FRIENDS_WITH]->(d)
               """)

In [ ]:
gph_conn.query("""
               CALL db.schema();
               """)

In [22]:
gph_conn.query("""
               PROFILE
               MATCH (u {id:"1"})
               RETURN u
               """)

[<Record u=<Node id=0 labels=frozenset({'User'}) properties={'id': '1'}>>]

In [ ]:
gph_conn.query("""
               PROFILE
               MATCH (u:User {id:"1"})
               RETURN u
               """)

In [23]:
gph_conn.query("""
               CREATE INDEX ON :User(id)
               """)

[]

In [ ]:
gph_conn.query("""
               PROFILE
               MATCH (u:User {id:"1"})
               RETURN u
               """)

In [24]:
gph_conn.query("""
               MATCH (a:User {id:"3"})-[:IS_FRIENDS_WITH]->(b:User)
               RETURN COUNT(b.id)
               """)

[<Record COUNT(b.id)=6>]

In [ ]:
gph_conn.query("""
               EXPLAIN
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MATCH (s:User {id: row.source_node}) 
               MATCH (d:User {id: row.destination_node}) 
               MERGE (s)-[:IS_FRIENDS_WITH]->(d)
               """)

In [9]:
gph_conn.query("""
               CALL apoc.load.json("neo4j_repos_github.json")
               YIELD value AS item 
               RETURN item 
               LIMIT 1
               """)

[<Record item={'stargazers_count': 3, 'pushed_at': '2019-01-15T04:05:09Z', 'subscription_url': 'https://api.github.com/repos/neo4j/parents/subscription', 'language': None, 'branches_url': 'https://api.github.com/repos/neo4j/parents/branches{/branch}', 'issue_comment_url': 'https://api.github.com/repos/neo4j/parents/issues/comments{/number}', 'labels_url': 'https://api.github.com/repos/neo4j/parents/labels{/name}', 'subscribers_url': 'https://api.github.com/repos/neo4j/parents/subscribers', 'permissions': {'admin': False, 'pull': True, 'push': False}, 'releases_url': 'https://api.github.com/repos/neo4j/parents/releases{/id}', 'svn_url': 'https://github.com/neo4j/parents', 'id': 1315642, 'forks': 8, 'archive_url': 'https://api.github.com/repos/neo4j/parents/{archive_format}{/ref}', 'git_refs_url': 'https://api.github.com/repos/neo4j/parents/git/refs{/sha}', 'forks_url': 'https://api.github.com/repos/neo4j/parents/forks', 'statuses_url': 'https://api.github.com/repos/neo4j/parents/statuse

In [10]:
gph_conn.query("""
               CALL apoc.load.json("neo4j_repos_github.json")
               YIELD value AS item
               CREATE (
                       r:Repository {
                                     name: item.name
               ,                     created_at: item.created_at
               ,                     contributors_url: item.contributors_url
               })
               MERGE (u:User {login: item.owner.login})
               CREATE (u)-[:OWNS]->(r)
               """)

[]